In [1]:
import pandas as pd
import numpy as np
import geojson as gj

In [2]:
file_to_load = "../json_files/LA_Certificated_Dispensary_Locations.csv"
disp_df = pd.read_csv(file_to_load)

In [3]:
# dispURL = 'https://data.lacity.org/resource/bunu-zsyc.geojson'
# dispDF = pd.read_json(dispURL)

# dispDF.head()

In [4]:
disp_df.reset_index(drop = True, inplace = True)

In [5]:
disp_df[["lat","long"]] = disp_df.LOCATION.str.split(",",expand=True)

In [6]:
# disp_df.head()

In [7]:
# ********************DONT RUN MORE THAN ONCE OR YOU WILL NEED TO RESTART KERNAL
disp_df["lat"] = disp_df.lat.str.slice(start=1)
disp_df["lat"] = pd.to_numeric(disp_df["lat"])

In [8]:
# ********************DONT RUN MORE THAN ONCE OR YOU WILL NEED TO RESTART KERNAL
disp_df["long"] = disp_df.long.str.slice(stop=-1)
disp_df["long"] = pd.to_numeric(disp_df["long"])

In [18]:
# Create a year column so we can filter this in the dashboarc
disp_df["START YEAR"] = pd.DatetimeIndex(disp_df["LOCATION START DATE"]).year

In [19]:
# disp_df.head()

,LOCATION ACCOUNT #,BUSINESS NAME,DBA NAME,CONFIRMED,STREET ADDRESS,CITY,ZIP CODE,LOCATION DESCRIPTION,MAILING ADDRESS,MAILING CITY,MAILING ZIP CODE,COUNCIL DISTRICT,LOCATION START DATE,LOCATION,lat,long,START YEAR
0,0003073895-0001-0,PARADISE FACTORY LLC,NaN,NaN,385 W 20TH STREET,SAN PEDRO,90731-5503,385 20TH 90731-5503,NaN,NaN,NaN,15,1/1/2018,"(33.7258, -118.2855)",33.7258,-118.2855,2018
1,0003067561-0001-9,WALIBE SOLUTIONS INC,NaN,NaN,400 N MARINE AVENUE,WILMINGTON,90744-5629,400 MARINE 90744-5629,NaN,NaN,NaN,15,1/1/2018,"(33.7744, -118.2629)",33.7744,-118.2629,2018
2,0002201665-0001-0,DELTA-9 T.H.C.,DELTA-9 T.H.C,NaN,824 E ANAHEIM STREET UNIT #B,WILMINGTON,90744-6833,824 ANAHEIM 90744-6833,824 E ANAHEIM STREET UNIT #B,WILMINGTON,90744-6833,15,12/1/2006,"(33.7805, -118.2523)",33.7805,-118.2523,2006
3,0002275038-0001-7,KUSH KORNER II,K.K.II,NaN,813 CRISTOBAL AVENUE,WILMINGTON,90744-4708,813 CRISTOBAL 90744-4708,10058 SUNLAND BLVD,SUNLAND,91040-1649,15,9/13/2007,"(33.7816, -118.243)",33.7816,-118.2430,2007
4,0002247121-0001-5,HERBAL SOLUTIONS SOUTHERN CALIFORNIA INC,EVOLV CANNABIS | LA BREA COMPASSIONATE CAREGIVERS,NaN,25937 WESTERN AVENUE,LOMITA,90717-3361,25937 WESTERN 90717-3361,25937 WESTERN AVENUE,LOMITA,90717-3361,15,5/15/2007,"(33.7871, -118.3081)",33.7871,-118.3081,2007


In [10]:
# disp_df.to_json(r'../JSON Files/DISP.JSON', orient = "index")
disp_df.dtypes

LOCATION ACCOUNT #       object
BUSINESS NAME            object
DBA NAME                 object
CONFIRMED                object
STREET ADDRESS           object
CITY                     object
ZIP CODE                 object
LOCATION DESCRIPTION     object
MAILING ADDRESS          object
MAILING CITY             object
MAILING ZIP CODE         object
COUNCIL DISTRICT          int64
LOCATION START DATE      object
LOCATION                 object
lat                     float64
long                    float64
dtype: object

In [20]:
def data2geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            gj.Feature(geometry=gj.Point((X["long"],
                                          X["lat"]
#                                           ,X["elev"]
                                         )),
                            properties=dict(zip_code=X["ZIP CODE"],
                                            business_name = X["BUSINESS NAME"],
                                            street_address = X["STREET ADDRESS"],
                                            location_start_date = X["LOCATION START DATE"],
                                            location_start_year = X["START YEAR"])))
    df.apply(insert_features, axis=1)
    with open(filename, 'w', encoding='utf8') as fp:
        gj.dump(gj.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

In [21]:
outputFilePath = '../static/data/'
dispFileName = f'{outputFilePath}disp.geojson'
data2geojson(disp_df, dispFileName)